## Autorzy: *Bartłomiej Sadza*, *Mateusz Strojek*
## Kierunek: *Informatyka i Ekonometria (1 rok stacjonarnie, 2 stopień)*
## Przedmiot: *Machine Learning*
## Sprawozdanie nr 1.

## Wstępna analiza i oczyszczenie zmiennych

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

data = pd.read_csv(
    "Stroke_data.csv",
    sep=";",
    decimal=",",
)
print(data.head())
print(data.dtypes)

   gender age  hypertension  heart_disease ever_married      work_type  \
0    Male  67             0              1          Yes        Private   
1  Female  61             0              0          Yes  Self-employed   
2    Male  80             0              1          Yes        Private   
3  Female  49             0              0          Yes        Private   
4  Female  79             1              0          Yes  Self-employed   

  Residence_type avg_glucose_level   bmi   smoking_status  stroke  
0          Urban            228.69  36.6  formerly smoked       1  
1          Rural            202.21   NaN     never smoked       1  
2          Rural            105.92  32.5     never smoked       1  
3          Urban            171.23  34.4           smokes       1  
4          Rural            174.12    24     never smoked       1  
gender               object
age                  object
hypertension          int64
heart_disease         int64
ever_married         object
work_ty

### Można wyróżnić poszczególne zmienne:

- *gender* : Płeć pacjenta (Male - mężczyzna, Female - kobieta),
- *age* : Wiek pacjenta,
- *hypertension* : Czy pacjent ma nadciśnienie (0 - brak, 1 - obecność nadciśnienia),
- *heart_disease* : Czy pacjent ma chorobę serca (0 - brak, 1 - obecność choroby serca),
- *ever_married* : Czy pacjent był kiedykolwiek w związku małżeńskim,
- *work_type* : Typ pracy wykonywanej przez pacjenta,
- *Residence_type* : Miejsce zamieszkania pacjenta,
- *avg_glucose_level* : Średni poziom glukozy we krwi pacjenta,
- *bmi* : Wskaźnik masy ciała (Body Mass Index, BMI) pacjenta,
- *smoking_status* : Status palenia pacjenta,
- *stroke* : Czy pacjent przeszedł udar

### Zmienna age, avg_glucose_level i bmi mają typ "object", co jest dość zastanawiające. Należy sprawdzić, dlaczego tak jest.

#### - Zmienna *Age*

In [4]:
print(data['age'].value_counts())

age
78      102
57       95
52       90
54       87
51       86
       ... 
1.4       3
0.48      3
0.16      3
0.4       2
0.08      2
Name: count, Length: 104, dtype: int64


#### - Zmienna *avg_glucose_level*

In [5]:
print(data['avg_glucose_level'].value_counts())

avg_glucose_level
93.88     6
91.68     5
91.85     5
83.16     5
73        5
         ..
94.07     1
111.93    1
94.4      1
95.57     1
85.28     1
Name: count, Length: 3978, dtype: int64


#### - Zmienna *bmi*

In [6]:
data['bmi'].value_counts()

bmi
28.7    41
28.4    38
26.7    37
27.6    37
26.1    37
        ..
48.7     1
49.2     1
51       1
49.4     1
14.9     1
Name: count, Length: 418, dtype: int64

### Zmienne posiadają taką wadę, że w pliku CSV zamiast wartości liczbowych, są wartości typu Date. Trzeba zamienić te zmienne na typ liczbowy, a na dodatek dla zmiennej *age* wyrzucić te obserwacje, które przyjmują wartości zmiennoprzecinkowe

In [7]:
data['age'] = pd.to_numeric(data['age'], errors='coerce')
data = data[(data['age'] % 1 == 0)]
data['avg_glucose_level'] = pd.to_numeric(data['avg_glucose_level'], errors='coerce')
data['bmi'] = pd.to_numeric(data['bmi'], errors='coerce')

### Z pomocą one-hot encoding zamieniamy zmienne kategoryczne na zmienne przyjmujące wartości zero jedynkowe.

In [8]:
categorical_columns = [
    "gender",
    "ever_married",
    "work_type",
    "Residence_type",
    "smoking_status",
]
data = pd.get_dummies(data, columns=categorical_columns)
data

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,ever_married_No,ever_married_Yes,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,False,True,False,True,...,False,True,False,False,False,True,False,True,False,False
1,61.0,0,0,202.21,NaN,1,True,False,False,True,...,False,False,True,False,True,False,False,False,True,False
2,80.0,0,1,105.92,32.5,1,False,True,False,True,...,False,True,False,False,True,False,False,False,True,False
3,49.0,0,0,171.23,34.4,1,True,False,False,True,...,False,True,False,False,False,True,False,False,False,True
4,79.0,1,0,174.12,24.0,1,True,False,False,True,...,False,False,True,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,80.0,1,0,83.75,NaN,0,True,False,False,True,...,False,True,False,False,False,True,False,False,True,False
5105,81.0,0,0,125.20,40.0,0,True,False,False,True,...,False,False,True,False,False,True,False,False,True,False
5106,35.0,0,0,82.99,30.6,0,True,False,False,True,...,False,False,True,False,True,False,False,False,True,False
5107,51.0,0,0,166.29,25.6,0,False,True,False,True,...,False,True,False,False,True,False,False,True,False,False


### Zastąpienie pustych wartości średnią dla zmiennych liczbowych oraz wartością najczęściej występującą dla zmiennych kategorycznych

In [9]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns

# Zastąpienie NA wartościami średnimi
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Zastąpienie NA wartością najczęstszą
for column in non_numeric_columns:
    data[column] = data[column].fillna(data[column].mode()[0])

X = data.drop(columns=["stroke"])
y = data["stroke"]

# **Zadanie 1. Pytania wstępne.**

## Zad. 1.1. W jakim celu dokonuje się podziału zbioru danych na zbiór uczący i testowy? W jaki sposób ten podział powinien zostać wykonany?

### Podział zbioru danych na zbiór uczący i testowy

Podział zbioru danych na zbiór uczący i testowy jest kluczowym krokiem w procesie tworzenia modeli uczenia maszynowego. Oto główne cele i sposób wykonania tego podziału:

### Cele podziału zbioru danych:
1. **Ocena modelu**: Podział danych pozwala na ocenę modelu na danych, które nie były używane podczas jego trenowania. Dzięki walidacji, można zobaczyć, jak dobrze poradzi sobie model na nowych danych.
2. **Unikanie przeuczenia (overfitting)**: Trenowanie modelu na całym zbiorze danych może prowadzić do przeuczenia, gdzie model dobrze radzi sobie na danych treningowych, ale słabo na nowych, niewidzianych danych. 

### Sposób wykonania podziału:
**Losowy podział** - Najczęściej stosowaną metodą jest losowy podział danych na zbiór uczący i testowy. Typowy podział to 80% danych na zbiór uczący i 20% na zbiór testowy, ale proporcje mogą się różnić w zależności od wielkości zbioru danych i specyfiki problemu.


## Zad. 1.2. Czym jest macierz błędu?

W przypadku klasyfikacji binarnej, jest to macierz 2x2, w której na podstawie problemu klasyfikacji pacjentów chorzych na COVID, można wyróżnić poszczególne komórki:
- TP (True Positive) - liczba pacjentów, którzy zostali sklasyfikowani jako chorzy i faktycznie byli chorzy,
- FP (False Positive) - liczba pasjentów, którzy zostali sklasyfkowani jako chorzy lecz byli zdrowi,
- FN (False Negative) - liczba pacjentów, którzy zostali sklasyfikowani jako zdrowi lecz byli chorzy,
- TN (True Negative) - liczba pacjentów, którzy zostali sklasyfikowani jako zdrowi i byli zdrowi.

## Zad. 1.3. Jaka jest różnica między dokładnością, czułością a specyficznością? Która z tych miar i w jakim przypadku jest ważniejsza. Podać przykłady.

1. **Dokładność (Accuracy)**: Jest to miara, która określa, jaki procent wszystkich przewidywań modelu jest poprawny. Oblicza się ją jako stosunek liczby poprawnych przewidywań do całkowitej liczby przypadków. Dokładność jest użyteczna, gdy klasy są zrównoważone, czyli liczba przypadków każdej klasy jest podobna.
   - Przykład: W klasyfikacji emaili jako spam lub nie-spam, jeśli mamy 1000 emaili, z czego 950 to nie-spam i 50 to spam, model, który zawsze przewiduje "nie-spam", będzie miał dokładność 95%, ale nie będzie użyteczny w wykrywaniu spamu.
2. **Czułość (Sensitivity)**: Jest to miara, która określa, jaki procent rzeczywistych pozytywnych przypadków został poprawnie zidentyfikowany przez model. Oblicza się ją jako stosunek liczby prawdziwych pozytywnych przewidywań do liczby wszystkich rzeczywistych pozytywnych przypadków. Czułość jest ważna w sytuacjach, gdzie istotne jest wykrycie wszystkich pozytywnych przypadków.
   - Przykład: W diagnostyce medycznej, gdzie chcemy wykryć wszystkie przypadki choroby, wysoka czułość jest kluczowa, aby zminimalizować ryzyko przeoczenia chorego pacjenta.
3. **Specyficzność (Specificity)**: Jest to miara, która określa, jaki procent rzeczywistych negatywnych przypadków został poprawnie zidentyfikowany przez model. Oblicza się ją jako stosunek liczby prawdziwych negatywnych przewidywań do liczby wszystkich rzeczywistych negatywnych przypadków.
   - Przykład: W systemach bezpieczeństwa, gdzie fałszywe alarmy mogą być kosztowne i uciążliwe, wysoka specyficzność jest kluczowa, aby zminimalizować liczbę fałszywych alarmów.

# **Zadanie 2. Metoda KNN**

## Zad. 2.1. Jak działa metoda k-najbliższych sąsiadów?
Metoda k-najbliższych sąsiadów (knn) pomaga analitykowi danych w klasyfikacji obserwacji na podstawie najbliższych k obserwacji. Odległość może być mierzona w różny sposób, jedną z najpopularniejszych metod jest metoda odległości euklidesowej. Na podstawie tych odległości, za pomocą k najbliższych sąsiadów głosów zapada decyzja odnośnie przynależności obserwacji dla danej klasy. Większość "głosów" sąsiadów decyduje o finalnej decyzji. 
## Zad. 2.2. Czym jest k w metodzie KNN? Jak dobrać odpowiednią wartość k?
K w metodzie KNN jest parametrem, który odpowiada za liczbę najbliższych sąsiadów branych pod uwagę przy decyzji klasyfikacji danej obserwacji. Wartość k jest optymalna, gdy np. pole AUC jest największe dla danych testowych, bądź inne kryterium, takie jak największa dokładność, czułość lub specyficzność
## Zad. 2.3. Czy k powinno być liczbą parzystą, nieparzystą, czy nie ma to znaczenia? Odpowiedź uzasadnić przeprowadzając odpowiednią symulację.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

czulosc = []
dokladnosc = []

for x in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=x)
    knn.fit(X_train_scaled, y_train)
    
    y_test_prediction = knn.predict(X_test_scaled)
    
    matrix_test = confusion_matrix(y_test, y_test_prediction)
    dokladnosc.append((matrix_test[0,0]+matrix_test[1,1])/matrix_test.sum())
    czulosc.append((matrix_test[1,1]/(matrix_test[1,0]+matrix_test[1,1])))

df = pd.DataFrame({
    'k': [x for x in range(1, 11)],
    'czulosc': czulosc,
    'dokladnosc': dokladnosc
})
print(df)

    k   czulosc  dokladnosc
0   1  0.083333    0.906907
1   2  0.033333    0.938939
2   3  0.033333    0.928929
3   4  0.016667    0.939940
4   5  0.050000    0.940941
5   6  0.033333    0.940941
6   7  0.033333    0.940941
7   8  0.033333    0.940941
8   9  0.033333    0.940941
9  10  0.000000    0.939940


Widać, że gdy k jest liczbą nieparzystą, wyniki są minimalnie lepsze od tych gdy jest liczbą parzystą. Może to wynikać z faktu, że gdy mamy remis w głosowaniu, to klasyfikacja jest przeprowadzana losowo.

## Zad. 2.4. Czy standaryzacja danych jest wymagana w przypadku wykorzystywania metody k-najbliższych sąsiadów? Dlaczego tak/nie? Zastosować metodę KNN na danych bez standaryzacji i ze standaryzacją. Porównać uzyskane wyniki.


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dokladnosc = []
czulosc = []

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

for x in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=x)
    
    knn.fit(X_train, y_train)
    
    y_test_prediction = knn.predict(X_test)
        
    matrix_test = confusion_matrix(y_test, y_test_prediction)
    dokladnosc.append((matrix_test[0,0]+matrix_test[1,1])/matrix_test.sum())
    czulosc.append((matrix_test[1,1]/(matrix_test[1,0]+matrix_test[1,1])))
    
df = pd.DataFrame({
    'k': [x for x in range(1, 11)],
    'czulosc': czulosc,
    'dokladnosc': dokladnosc
})
print(df)

    k   czulosc  dokladnosc
0   1  0.133333    0.907908
1   2  0.016667    0.934935
2   3  0.066667    0.929930
3   4  0.000000    0.936937
4   5  0.016667    0.931932
5   6  0.000000    0.936937
6   7  0.016667    0.936937
7   8  0.000000    0.937938
8   9  0.000000    0.937938
9  10  0.000000    0.938939


Standaryzacja jest zalecana w przypadku metody KNN. Jeżeli np. jedna zmienna przyjmuje wartości od 1 do 500 a reszta ma o wiele mniejszy zakres, to macierz odległości zależy w większości od zmiennej, której wartości mają bardzo szeroki zakres. Wyniki wyszły bardzo podobne, choć czułość wypadła jeszcze gorzej niż w przypadku standaryzowania zmiennych.

## Zad. 2.5. Czy wielkość zbioru danych ma znaczenie w przypadku tej metody? Sprawdzić, czy liczba obserwacji wpływa na uzyskiwane wyniki.

In [12]:
czulosc = []
dokladnosc = []

for x in range(1,11):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05+x*0.05, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=x)
    knn.fit(X_train_scaled, y_train)
    
    y_test_prediction = knn.predict(X_test_scaled)
    
    matrix_test = confusion_matrix(y_test, y_test_prediction)
    dokladnosc.append((matrix_test[0,0]+matrix_test[1,1])/matrix_test.sum())
    czulosc.append((matrix_test[1,1]/(matrix_test[1,0]+matrix_test[1,1])))

df = pd.DataFrame({
    'próbka testowa': [0.05+x*0.05 for x in range(1, 11)],
    'czulosc': czulosc,
    'dokladnosc': dokladnosc
})
print(df)

   próbka testowa   czulosc  dokladnosc
0            0.10  0.090909    0.892000
1            0.15  0.043478    0.938667
2            0.20  0.033333    0.928929
3            0.25  0.013158    0.939151
4            0.30  0.023256    0.941294
5            0.35  0.020202    0.943936
6            0.40  0.036036    0.945946
7            0.45  0.008403    0.947509
8            0.50  0.015625    0.949539
9            0.55  0.000000    0.948671


Wielkość próbki ma znaczenie, w tym przypadku wniosek jest taki, że im większa próbka testowa, tym większa dokładność oraz mniejsza czułość.

## Zad. 2.6. Czy w metodzie tej można wykorzystać i jeśli tak, to w jaki sposób zmienne kategoryczne?

Tak, można wykorszystywać w tej metodzie dane kategoryczne. Należy zamienić je na wartości binarne. Dla zmiennej kategorycznej przyjmującej n wartości należy zbudować n-1 kolumn przyjmujących wartości 1 lub 0.

## Zad. 2.7. Jakie są zalety i wady tej metody?

Główną zaletą metody KNN jest prostota w implementacji oraz zrozumieniu, klasyfikacja odbywa się na podstawie k najbliższych sąsiadów. Z drugiej strony, metoda jest podatna na brakujące wartości, złożoność obliczeniowa jest spora oraz metoda jest podatna na wartości odstające.

# **Zadanie 3. Metoda KKNN.**

## Zad.3.1. Co odróżnia metodę KKNN od metody KNN?

Metoda KKNN (Kernel K-Nearest Neighbors) różni się od klasycznej metody KNN (K-Nearest Neighbors) tym, że wprowadza wazenie sąsiadów. W klasycznym KNN każdy z k najbliższych sąsiadów ma równy wpływ na decyzję, podczas gdy w KKNN sąsiedzi bliżsi mają większy wpływ niż dalsi.

## Zad. 3.2. Czy w metodzie tej występują ograniczenia dotyczące wyboru wartości k?

Tak, podobnie jak w klasycznym KNN, wybór wartości k w KKNN jest istotny. Zbyt mała wartość k może prowadzić do nadmiernego dopasowania (overfitting), podczas gdy zbyt duża wartość k może prowadzić do niedopasowania (underfitting). Wartość k powinna być dobrana w taki sposób, aby zbalansować te dwa zjawiska.

## Zad. 3.3. Czy wielkość zbioru danych ma znaczenie w przypadku tej metody? Sprawdzić, czy liczba obserwacji wpływa na uzyskiwane wyniki.

In [13]:
czulosc = []
dokladnosc = []

for x in range(1,11):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05+x*0.05, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=x, weights="distance")
    knn.fit(X_train_scaled, y_train)
    
    y_test_prediction = knn.predict(X_test_scaled)
    
    matrix_test = confusion_matrix(y_test, y_test_prediction)
    dokladnosc.append((matrix_test[0,0]+matrix_test[1,1])/matrix_test.sum())
    czulosc.append((matrix_test[1,1]/(matrix_test[1,0]+matrix_test[1,1])))

df = pd.DataFrame({
    'próbka testowa': [0.05+x*0.05 for x in range(1, 11)],
    'czulosc': czulosc,
    'dokladnosc': dokladnosc
})
print(df)

   próbka testowa   czulosc  dokladnosc
0            0.10  0.090909    0.892000
1            0.15  0.086957    0.892000
2            0.20  0.033333    0.923924
3            0.25  0.039474    0.932746
4            0.30  0.034884    0.939293
5            0.35  0.040404    0.942220
6            0.40  0.036036    0.943944
7            0.45  0.033613    0.945730
8            0.50  0.031250    0.947537
9            0.55  0.021429    0.948307


Tak jak w metodzie KNN, dla tego zestawu danych czułość się zmniejsza, natomiast dokładność rośnie. Nie jest to jednak pożądany efekt, ponieważ celem jest wykrycie śmiertelnej choroby.